In [1]:
import cv2
import sys
import time
import numpy as np
import os

In [25]:
def loadTrainImageSet(add):
    trainingImages = []
    trainingIndices = []
    for i in os.listdir(add):
        try:
            img = cv2.imread(add+i,cv2.IMREAD_GRAYSCALE)
            img = cv2.equalizeHist(img)
            trainingImages.append(np.asarray(img))
#             number = filter(str.isdigit, i)
            number = i.split('_')[0]
            trainingIndices.append(int(number))             
        except:
            print 'load %s failed'%i
    return trainingImages, trainingIndices

def trainRecognizer(trainingImages,trainingIndices, modelType = 0):
    if modelType == 0:
        recognizer = cv2.createLBPHFaceRecognizer()
        print "LBPH selected..."
    elif modelType == 1:
        recognizer = cv2.createFisherFaceRecognizer()
        print "Fisher selected..."
    elif modelType == 2:
        recognizer = cv2.createEigenFaceRecognizer()
        print "Eigen selected..."
    recognizer.train(np.asarray(trainingImages, dtype=np.uint8), np.asarray(trainingIndices, dtype=np.int))
    return recognizer

trainingImages, trainingIndices = loadTrainImageSet(path)
recognizer = trainRecognizer(trainingImages,trainingIndices)


LBPH selected...
1 2.43184207934


In [3]:
def drawBlock(faces,img):
    for (x, y, w, h) in faces:
        rec = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 0),thickness=cv2.cv.CV_FILLED)
    return img

def drawLable(faces,img,ratio,text):

    height,width,col = img.shape
    for (x,y,w,h) in faces:       
        x *= int(1/ratio)
        y *= int(1/ratio)
        w *= int(1/ratio)
        h *= int(1/ratio)
        # Draw a label with text below the face
        font = cv2.FONT_HERSHEY_DUPLEX
        # draw rectangle around face
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # draw label around face
        cv2.rectangle(img, (x, y), (x+w, y+20), (0, 255, 0), thickness=cv2.cv.CV_FILLED)
        cv2.putText(img, text, (x , y + 15 ), font, 0.7, (255, 255, 255), 1)
          
    return img
        
def frontFaceDetect(img):
    cascPath = 'C:/Users/VE-ENG/Desktop/data/haarcascades/haarcascade_frontalface_alt2.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(
        img,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.cv.CV_HAAR_SCALE_IMAGE)
    return faces
    
def sideFaceDetect(img, flip = 0):
    cascPath = 'C:/Users/VE-ENG/Desktop/data/haarcascades/haarcascade_profileface.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)
    if flip == 1:
        img = cv2.flip(img,1)
        
    faces = faceCascade.detectMultiScale(
        img,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.cv.CV_HAAR_SCALE_IMAGE)
    
    if len(faces) != 0 and flip == 1:
        x,y,w,h = faces[0][:]
        height,width,col = img.shape
        x,y = mirrorCoordinate(x,y,width,height)
        faces = [[x-w,y,w,h]]
    return faces

def mirrorCoordinate(x,y,w,h):
    # center line: x = w/2  ==> Ax+By+C =0 ==> A=1, B=0, C=-w/2
    # http://www.voidcn.com/blog/cuixiping/article/p-813971.html
    A,B,C = 1,0,-w/2
    trans = (1/(A**2+B**2))*np.mat([[B**2-A**2, -2*A*B, -2*A*C],
                                   [-2*A*B, A**2-B**2, -2*B*C],
                                    [0, 0, 1]
                                   ])
    original = np.mat([x,y,1]).T
    res = np.dot(trans,original)
    return int(res[0]), int(res[1])

def corpFace(faces,img,ratio,default_height = 100,default_width = 100):
    if len(img.shape) == 3:
        height,width,col = img.shape
    else:
        height,width = img.shape
    face_list = []
    for (x,y,w,h) in faces:
        x *= int(1/ratio)
        y *= int(1/ratio)
        w *= int(1/ratio)
        h *= int(1/ratio)
        face = img[y:y+w,x:x+h]
        face = cv2.resize(face, (default_height,default_width), interpolation=cv2.INTER_CUBIC)
        if len(img.shape) == 3:
            face =  cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = cv2.equalizeHist(face)
#         face_list = np.append(face_list,face)
    return face if len(faces) != 0 else np.zeros((default_height,default_width))

In [24]:
video_capture = cv2.VideoCapture(0)
ratio = .25
threshould = 100
process_this_frame = True

# Train facial recognizer
path = './FaceDatabase/'
trainingImages, trainingIndices = loadTrainImageSet(path)
recognizer = trainRecognizer(trainingImages,trainingIndices, modelType= 0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    # Resize frame of video to 1/4 size for faster face recognition processing
    if ret:
        small_frame = cv2.resize(frame, (0, 0), fx=(ratio), fy=(ratio))
    else:
        print 'WebCam Error!'

    # Only process every other frame of video to save time
    if process_this_frame:
        
        small_frame_new = small_frame.copy()
        # Find all the faces in the current frame of video
        front_face = frontFaceDetect(small_frame)
        small_frame_new = drawBlock(front_face,small_frame_new)
        
        side_face = sideFaceDetect(small_frame_new)
        small_frame_new = drawBlock(side_face,small_frame_new)
        
        side_flip_face = sideFaceDetect(small_frame_new, flip = 1)
        small_frame_new = drawBlock(side_flip_face,small_frame_new)
        cv2.imshow('Blocked', small_frame_new)
   
    
    f1 = corpFace(front_face,frame,ratio)
    f2 = corpFace(side_face,frame,ratio)
    f3 = corpFace(side_flip_face,frame,ratio)
    
    prediction, confidence = recognizer.predict(f1)
    prediction = prediction if confidence <=threshould else 'Unknow'
    frame = drawLable(front_face, frame, ratio, str(prediction)+ ' ' + str(int(confidence)))
    prediction, confidence = recognizer.predict(f2)
    prediction = prediction if confidence <=threshould else 'Unknow'
    frame = drawLable(side_face, frame, ratio, str(prediction)+ ' ' + str(int(confidence)))
    prediction, confidence = recognizer.predict(f3)
    prediction = prediction if confidence <=threshould else 'Unknow'
    frame = drawLable(side_flip_face, frame, ratio, str(prediction)+ ' ' + str(int(confidence)))
    
#     cv2.imshow('front',f1)
#     cv2.imshow('side',f2)
#     cv2.imshow('flip side',f3)
       
    cv2.imshow('Video',frame)

    process_this_frame = not process_this_frame
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

LBPH selected...


In [26]:
# def prepareDatabase(path,faceName):
#     count = 0
#     for fileName in os.listdir(path) :
#         if '.jpg' in fileName:
#             print fileName
#             img = cv2.imread(path+fileName,0)
#             face1 = frontFaceDetect(img)
#             face2 = sideFaceDetect(img)
#             face3 = sideFaceDetect(img,flip=1)
#             if len(face1) != 0:
#                 corp_img = corpFace(face1,img, default_height= 100, default_width= 100)
#             elif len(face2) != 0:
#                 corp_img = corpFace(face2,img, default_height= 100, default_width= 100)
#             elif len(face3) != 0:
#                 corp_img = corpFace(face3,img, default_height= 100, default_width= 100)
#             else:
#             # no face found in the imge, skip the if loop
#                 continue
#             print face1, face2, face3
#             count += 1
#             cv2.imwrite(path + faceName + '_' + str(count) +'.jpg', corp_img)

# path = 'C:/Users/VE-ENG/Pictures/Camera Roll/'
# faceName = '0'
# prepareDatabase(path,faceName)       